# Display Nearest River results

---------------------------------------------------------------------------

<font face="cambria">
<u>Usage</u>: Display results for Get Distance to Nearest River (NEAR_DIST) for building centroids in a state, list of states, <br>
or each state in a FEMA region. Results can be displayed in a histogram with statistics in a text box, or in a CSV

<u>Python version</u>: 3.6

In [15]:
## Import libraries

In [16]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.dates as mdates
import datetime
from time import time
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import AutoMinorLocator
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import os
# path up to big-geodata folder
import sys; sys.path.append('../../')
import re
from glob import glob
from datetime import date
# within big-geodata go to core, and within core find ConventionOps.py and
    # displayNearestRiverResults.py
from core.ConventionOps import state_codes, stateabb, regstates, sfha_fld_zones
from core.near_analysis.displayNearestRiverResults import *

## User-defined variables

In [17]:
# Select Regions to Process
regions = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# regions = [4, 5, 6]

processingStateList = [statename for reggroup in [regstates[str(r)] for r in regions] for statename in reggroup]

In [18]:
# Mapped or Unmapped and Date of Nearest River CSVs
stream_type = 'Unmapped'
last_updated = '20190819'

# True = within SFHA only; False = all centroids
sfha = False

# histograms, tables, hist_and_tables
output_types = 'histograms'

# True = best fit text box; Fales = default upper right
best_fit = False

# folder containing your inputs
rootDir = r'T:\CCSI\TECH\FEMA\FEMA_HQ_Queries\FEMA_Mission_Support\Tier0_analytics'

# building attribute / flood zone inputs
bldg_attr_Root = r'T:\CCSI\TECH\FEMA\FEMA_HQ_Queries\FEMA_Mission_Support\Tier0_analytics\Building_attributes'

In [19]:
# Auto-populated

inDist_Folder = os.path.join(rootDir, f'NearFeatures\\{stream_type}_REDO_{last_updated}')
assert os.path.exists(inDist_Folder), 'Near folder does not exist, check inputs'

inCSV_Folder = os.path.join(inDist_Folder, f'{stream_type}_Streams_Dist_CSVs') 
assert os.path.exists(inCSV_Folder), 'Near CSVs folder does not exist, check inputs'

# folder for your outputs
outHist_Folder = os.path.join(inDist_Folder, 'histograms')
outTable_Folder = os.path.join(inDist_Folder, 'tables')

## Setup

In [20]:
# start timer
starttime = time()
# list of input files containing near distance and SFHA information
inDist_files = glob(os.path.join(inCSV_Folder, '*_DIST.csv'))
assert len(inDist_files) > 0, 'No Near dist CSVs exist! Add or change path'

inSFHA_files = glob(os.path.join(bldg_attr_Root, '*_Bldg_Attr.txt'))
assert len(inSFHA_files) > 0, 'No Building attribute CSVs (with flood zones) exist! Add or change path'

# folders for histogram and table outputs
if not os.path.exists(outHist_Folder): 
    os.mkdir(outHist_Folder)
if not os.path.exists(outTable_Folder): 
    os.mkdir(outTable_Folder)
    
# set up identifier for mapped or unmapped lines
if stream_type == 'Mapped':
    map_unmapped = 'ML'
else:
    map_unmapped = 'UML'
    # Alaska and Hawaii have no Unmapped streams
    if 'Alaska' in processingStateList: processingStateList.remove('Alaska')
    if 'Hawaii' in processingStateList: processingStateList.remove('Hawaii')
    
# set best fit and sfha
if best_fit: 
    bestFit = 'bestfit'
else: 
    bestFit = 'upperright'
    
    
if sfha: 
    insfha = 'sfha'
else: 
    insfha = 'all'

# location of the output histogram PDF (contains all the input states)
datestamp = str(date.today()).replace('-','')
newfileName = f'_{map_unmapped}_{insfha}_{bestFit}_{datestamp}'

pdfOut = os.path.join(outHist_Folder, 'histograms' + newfileName +'.pdf')

# location of the output table CSV (contains all the input states)
tableOut = os.path.join(outTable_Folder, 'tables' + newfileName + '.csv')

# sort the list of states alphabetically
processingStateList = sorted(processingStateList)

## Create one pdf of histograms and/or one CSV table for the specified states, arranged alphabetically by state

In [21]:
# for histograms
%matplotlib inline
# turn interactive mode off (for use with PdfPages)
plt.ioff()
# for tables
outdf = initializeTable(FIELDNAMES)
# create dummy histogram pdf output if only tables are output
if output_types == 'tables': 
    pdfOut = r'C:\temp\dummyHist.pdf'
with PdfPages(pdfOut) as pp:
    # loop through the states
    for st in processingStateList:
        print(st)
        st_abb = stateabb[st]
        st_fips = state_codes[st_abb]

        # read in the distance CSV for a state
        dist_df = readDistances(inCSV_Folder, st_abb, st_fips, map_unmapped, INDIST_FIELDS)

        print('All: ', len(dist_df))
        # if user specifies only centroids within SFHA, then read in building attributes CSV as data frame
        if sfha:
            bldg_attr_csv = os.path.join(bldg_attr_Root, st_fips+'_'+st_abb+'_Bldg_Attr.txt')
            bldg_attr_df = pd.read_csv(bldg_attr_csv, usecols=INSFHA_FIELDS)
            
            # retrieve only the building centroid ids within the SFHA
            dist_df = getQueriedData(dist_df, DIST_ID, DIST_FIELD, bldg_attr_df, BLDGATTR_ID, BLDGATTR_FLDZONE, sfha_fld_zones)
            print('Within SFHA: ', len(dist_df))
            
        # otherwise, get the Near distances for all of the centroids
        else:
            dist_df = dist_df[DIST_FIELD]

        meters_array = dist_df.values
        # remove nulls
        meters_array = meters_array[~np.isnan(meters_array)]
        # convert array to miles and get stats
        arrayStats = getStats(meters_array)

        # create histogram - write each state's histogram to the same PDF file using PdfPages
        if output_types =='histograms' : 
            createHistogram(arrayStats, sfha, st_abb, stateabb, pp, best_fit)
            
        elif output_types == 'tables': 
            outdf = outdf.append(addTableRow(outdf, FIELDNAMES, arrayStats, st, st_abb, st_fips), sort = False)
            
        elif output_types == 'hist_and_tables': 
            createHistogram(arrayStats, sfha, st_abb, stateabb, pp, best_fit)
            outdf = outdf.append(addTableRow(outdf, FIELDNAMES, arrayStats, st, st_abb, st_fips), sort = False)
            
# export all states to CSV
if output_types == 'tables': 
    outdf.to_csv(tableOut, index = False)
    os.remove(pdfOut)
elif output_types == 'hist_and_tables':
    outdf.to_csv(tableOut, index = False)
    
# end timer
elapsed = str(round(((time() - starttime) / 60), 2))
print(f'Completed in {elapsed} minutes')

Delaware
All:  331654
DistrictOfColumbia
All:  58330
Maryland
All:  1590655
Pennsylvania
All:  4801561
Virginia
All:  3020994
WestVirginia
All:  1020048
Completed in 0.84 minutes
